1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지못한다.
    - 문서가 길면(인풋이 길면) 답변 생성이 오래 걸린다.
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [6]:
#%pip install --upgrade pip
#%pip install langchain langchain-community langchain-upstage langchain-text-splitters python-dotenv docx2txt

Note: you may need to restart the kernel to use updated packages.
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3990 sha256=b918803a625e95f99165ece6f203e6e2aadddce7c361fb3cb2beefbfdf0f49f1
  Stored in directory: /Users/dot/Library/Caches/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter)

In [4]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [6]:
%pip install --upgrade --quiet \
    langchain-pinecone\
        langchain-upstage\
            langchain


Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name = 'tax-index'

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [22]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=3)

In [23]:
#retrieved_docs
from langchain_upstage import ChatUpstage
llm = ChatUpstage()

In [24]:
'''
prompt = f"""
[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""
'''

'\nprompt = f"""\n[Identity]\n- 당신은 최고의 한국 소득세 전문가입니다.\n- [Context]를 참고해서 사용자의 질문에 답변해주세요.\n\n[Context]\n{retrieved_docs}\n\nQuestion: {query}\n"""\n'

In [25]:
#ai_message = llm.invoke(prompt)
#ai_message.content

In [ ]:
#%pip install -U langchain langchainhub

In [26]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/dot/.pyenv/versions/3.10.16/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [30]:
from langchain.chains import RetrievalQA

retriever=database.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
retriever.invoke(query)

In [28]:
ai_message = qa_chain({"query": query})

/var/folders/k9/8dy1v0jj3v90yfb4ymql2n400000gn/T/ipykernel_46816/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [29]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '근로소득이 있는 거주자에 대해서는 그 근로소득에 대한 종합소득산출세액에서 근로소득공제액을 공제합니다. 해당 공제액은 총급여액에 따라 달라지며, 총급여액이 7천만원 초과 1억2천만원 이하인 경우에는 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만 원보다 적은 경우에는 50만원으로 합니다.'}